This notebook implements autoencoder trial run cycles.

* Define
    * trial name
    * environment to work with (MyBallsEnv, MyArmEnv, GymArmEnv)
    * dataset to use (set of 3 npz files for train, test and grid visualization, each file contains many angle/image pairs)
    * model builder
    * list of parameters to try
    * number of epochs to run 
    * output directory
* Run
    * fails if output directory exists, otherwise creates it
    * chooses a set of model parameters
Thursday 4/03/2021

* Visualize YYs (third output sheet or animated GIF) to assess decoder performance
* Rerun with [2, 10, 50] epochs
* Implement beta-VAE (see beta-vae notebook)

Next:

* Explore impact of changing filters chain down to 2x (2,2)
* Scatterplot 3D
* How does it fluctuate depending on network architecture, nlats, training protocol, etc
* Explore fold area in L0/L1 space. For each frame show:
 * image of gym robot,
 * colored L0/L1 scatterplot, with a red cross showing current latent state,
 * image reconstructed by the autoencoder

### imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt

%load_ext tensorboard

import numpy as np
import os, datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from src.config import output_data_dir

import logging
logger = logging.getLogger(__name__)


### helpers

In [2]:
def cycle_autoencoder(ae, dataset, prefix="", N=10, vae=False):
    Y = dataset['images']
    
    L = ae['enc'].predict(Y)
    L = np.array(L)

    if vae:
        L = np.array(L)[2] # [z_mean, z_log_var, z] - take Z

    YY = ae['dec'].predict(L)
    YY = np.array(YY)
    
    out_data = dict()
    out_data['latvars'] = L
    out_data['rec_images'] = YY

    return out_data    

### environment

In [3]:
import src.models.vae

# define ae builder method and environment name
ae_builder = src.models.vae.build_autoencoder
ENV_NAME = "twoballs"

### subdirectories

In [4]:
TRIAL_DIR = os.path.join(output_data_dir, ENV_NAME) # FUXNE

# fail if TRIAL_DIR exists
assert(not os.path.exists(TRIAL_DIR))
    
# create trial subdirs
TENSORBOARD_LOGS_DIR =  "%s/tensorboard-logs" % TRIAL_DIR
TRAINED_MODELS_DIR = "%s/trained-models" % TRIAL_DIR
#DATA_DIR = "%s/data" % TRIAL_DIR
IMGS_DIR = "%s/imgs" % TRIAL_DIR

#for dir in [TENSORBOARD_LOGS_DIR, TRAINED_MODELS_DIR, DATA_DIR, IMGS_DIR]:
for dir in [TENSORBOARD_LOGS_DIR, TRAINED_MODELS_DIR, IMGS_DIR]:
    os.makedirs(dir, exist_ok=True)

### load datasets

In [5]:
from src.data.dataset import load_datasets

datasets = load_datasets(ENV_NAME)

2021-03-07 10:43:02,112 - src.data.dataset - INFO - Loading from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_rand_15000.npz ...
2021-03-07 10:43:03,095 - src.data.dataset - INFO - Loaded 15000 datapoints from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_rand_15000.npz
2021-03-07 10:43:03,095 - src.data.dataset - INFO - Loading from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_rand_1000.npz ...
2021-03-07 10:43:03,161 - src.data.dataset - INFO - Loaded 1000 datapoints from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_rand_1000.npz
2021-03-07 10:43:03,161 - src.data.dataset - INFO - Loading from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_grid_20_500.npz ...
2021-03-07 10:43:04,472 - src.data.dataset - INFO - Loaded 20000 datapoints from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_grid_20_500.npz


### train

In [6]:
from src.models import save_models, save_models_weights
from src.visualization.lat import visualize_lat_space, animate_Y_YYs

def save_visualization(dataset, dataset_grid_out, round, epoch):
    vis_fname = "%s/round-%d_epoch-%d.png" % (IMGS_DIR, round, epoch)
    ani_fname = "%s/round-%d_epoch-%d.mp4" % (IMGS_DIR, round, epoch)

    # save 4x4 visualization of pairwise plots of latvars vs angles
    fig, axs = visualize_lat_space(dataset['grid'], dataset_grid_out, sheet=1)
    fig.savefig(vis_fname)
    
    #vis_fname2 = "%s/%sb.png" % (IMGS_DIR, suffix)
    #fig, axs = visualize_lat_space(dataset['grid'], dataset_grid_out, sheet=2)
    #fig.savefig(vis_fname2)

    # save MP4
    NANIFRAMES = 20
    logger.debug("saving ani visualization (for %d datapoints from grid dataset) into %s" % (NANIFRAMES, ani_fname))
    rng = np.random.default_rng()
    ani_is = rng.choice(range(dataset['grid']['images'].shape[0]), size=NANIFRAMES) # choose 20 random items from the dataset_grid
    animate_Y_YYs(dataset['grid']['images'][ani_is], dataset_grid_out['rec_images'][ani_is], outfile=ani_fname)

    plt.close('all')

def run_round(datasets, ae_builder, params, round):
    #if os.path.isfile(latvars_fname):
    #    logger.error("%s exists, skipping ..." % latvars_fname)
    #    return
    
    print("*** run_round(%d, %s)" % (round, str(params)))
       
    models = ae_builder((64, 64, 1), 2, params)

    tensorboard_logdir = os.path.join(("%s/round-%d" % (TENSORBOARD_LOGS_DIR, round)),
                                      datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = keras.callbacks.TensorBoard(tensorboard_logdir) #, histogram_freq=1)

    for epoch in range(params['epochs']):
        logger.info("round-%d_epoch-%d: fit" % (round, epoch))
        models['ae'].fit(datasets['train']['images'], callbacks=[tensorboard_callback],
               initial_epoch=epoch, epochs=epoch+1, batch_size=128)
        save_models_weights(models, "%s/round-%d_epoch-%d" % (TRAINED_MODELS_DIR, round, epoch))

        logger.info("round-%d_epoch-%d: cycle" % (round, epoch))
        dataset_grid_out = cycle_autoencoder(models, datasets['grid'], vae=True)
        save_visualization(datasets, dataset_grid_out, round, epoch)
    
        # save angles & latvars for grid dataset
        dataset_test_out = cycle_autoencoder(models, datasets['test'], vae=True)
        latvars_fname = "%s/round-%d_epoch-%d.csv" % (IMGS_DIR, round, epoch)
        angles_latvars = np.hstack((datasets['test']['angles'], dataset_test_out['latvars']))
        np.savetxt(latvars_fname, angles_latvars, delimiter=',')

In [ ]:
keras.backend.clear_session()
for round in range(10):
    run_round(datasets, ae_builder, {'epochs': 100}, round)

*** run_round(0, {'epochs': 100})


2021-03-07 10:43:06,113 - __main__ - INFO - round-0_epoch-0: fit


118/118 [==============================] - 7s 34ms/step - loss: 888.1096 - reconstruction_loss: 279.7002 - kl_loss: 44.7063


2021-03-07 10:43:12,976 - __main__ - INFO - round-0_epoch-0: cycle
2021-03-07 10:43:18,690 - __main__ - INFO - round-0_epoch-1: fit


Epoch 2/2
118/118 [==============================] - 4s 34ms/step - loss: 68.1296 - reconstruction_loss: 64.0024 - kl_loss: 2.8723


...

2021-03-07 10:58:27,913 - __main__ - INFO - round-0_epoch-83: cycle
2021-03-07 10:58:37,005 - __main__ - INFO - round-0_epoch-84: fit


Epoch 85/85
118/118 [==============================] - 4s 37ms/step - loss: 26.3771 - reconstruction_loss: 19.0889 - kl_loss: 7.1407: 1s - loss: 26.4533 - reconstruction_loss: 19


2021-03-07 10:58:41,674 - __main__ - INFO - round-0_epoch-84: cycle
2021-03-07 10:58:50,943 - __main__ - INFO - round-0_epoch-85: fit


Epoch 86/86
118/118 [==============================] - 4s 36ms/step - loss: 26.2934 - reconstruction_loss: 19.0453 - kl_loss: 7.1866


2021-03-07 10:58:55,445 - __main__ - INFO - round-0_epoch-85: cycle


### tensorboard

In [ ]:
%tensorboard --logdir ..\data\output\twoballs